In [1]:
%load_ext autoreload
%autoreload 2

import gc
import os
import sys
# import time
from pprint import pprint

# import googlemaps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from scipy.integrate import odeint

main_dir = os.path.abspath(os.pardir)
sys.path.insert(0, main_dir)
from analysis.download_data import jh_git_url, population_data_url, export_foldname, all_files
from covid_forecast.utils import data_prep as dp
from analysis.estimate_sir_params import estimate_sir_params

pd.options.display.max_rows = 999
pd.options.display.max_columns = 100
pd.options.display.precision = 8
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
export_dir = os.path.join(main_dir, "data", "raw"+os.sep)
countries_fname = "countries.csv"
print(main_dir+"\n"+export_dir)
province_state_vname = "Province/State"
country_region_vname = "Country/Region"
susceptible_vname = "s_0"
pop_df_vnames = [country_region_vname, susceptible_vname]

D:\Users\gjnet\code_projects\covid_forecast
D:\Users\gjnet\code_projects\covid_forecast\data\raw\


# Open files as dataframes
Credit to dgrechka for locations_population. Data [source](https://www.kaggle.com/dgrechka/covid19-global-forecasting-locations-population/metadata)

In [3]:
population_data = pd.read_excel(population_data_url)
friendly_fnames = [file.replace("time_series_covid19_", "").replace(".csv", "") for file in all_files]
filename_dct = dict(zip(friendly_fnames, all_files))
pprint(filename_dct)

{'confirmed_global': 'time_series_covid19_confirmed_global.csv',
 'deaths_global': 'time_series_covid19_deaths_global.csv',
 'recovered_global': 'time_series_covid19_recovered_global.csv'}


In [4]:
df_dct = dict()
for file in filename_dct.keys():
    df_dct[file] = pd.read_csv(jh_git_url+filename_dct[file])  # reads csv from github repo
    print(file, df_dct[file].shape)
    display(df_dct[file].head(), df_dct[file].describe(include="all"), df_dct[file].dtypes)

confirmed_global (254, 73)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20
0,NaN,Afghanistan,33.000,65.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74,84,94,110,110,120,170
1,NaN,Albania,41.153,20.168,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174,186,197,212,223
2,NaN,Algeria,28.034,1.660,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,74,87,90,139,201,230,264,302,367,409,454,511,584
3,NaN,Andorra,42.506,1.522,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,39,53,75,88,113,133,164,188,224,267,308,334,370
4,NaN,Angola,-11.203,17.874,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,4,4,5,7,7


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20
count,79,254,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000
unique,79,178,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
top,Northern Territory,China,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
freq,1,33,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
mean,NaN,NaN,21.976,23.270,2.185,2.575,3.705,5.646,8.339,11.524,21.961,24.276,32.417,39.083,47.394,66.091,78.272,94.063,108.799,121.236,135.398,146.142,158.071,168.354,176.386,178.035,237.669,263.327,271.772,280.409,288.417,295.811,297.791,299.988,302.437,309.339,310.858,313.232,316.559,320.425,325.772,331.150,338.626,347.909,355.535,365.512,374.488,385.378,400.791,416.720,432.366,447.205,467.008,495.571,505.323,571.673,614.571,659.268,714.858,775.992,845.752,954.724,"1,071.004","1,198.409","1,326.587","1,489.114","1,645.846","1,841.154","2,085.004","2,335.791","2,601.205","2,835.106","3,080.177"
std,NaN,NaN,24.515,71.366,27.915,28.014,34.873,48.532,67.820,91.374,224.277,225.635,310.945,368.649,453.648,705.047,852.318,"1,050.457","1,237.898","1,391.569","1,569.821","1,704.599","1,863.288","1,994.778","2,097.525","2,097.866","3,026.954","3,415.493","3,531.079","3,652.281","3,765.569","3,871.712","3,893.619","3,919.369","3,933.231","4,022.342","4,022.375","4,035.161","4,066.487","4,091.814","4,118.024","4,138.961","4,167.438","4,205.490","4,220.674","4,233.288","4,244.941","4,259.308","4,276.860","4,294.524","4,311.249","4,331.063","4,347.386","4,381.238","4,391.682","4,486.731","4,570.047","4,667.178","4,783.062","4,920.994","5,106.026","5,402.822","5,763.312","6,200.695","6,647.479","7,266.732","7,893.377","8,731.214","9,865.187","11,048.735","12,351.750","13,572.269","14,905.608"
min,NaN,NaN,-41.455,-135.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,NaN,NaN,8.094,-18.093,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0

Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
1/23/20             int64
1/24/20             int64
1/25/20             int64
1/26/20             int64
1/27/20             int64
1/28/20             int64
1/29/20             int64
1/30/20             int64
1/31/20             int64
2/1/20              int64
2/2/20              int64
2/3/20              int64
2/4/20              int64
2/5/20              int64
2/6/20              int64
2/7/20              int64
2/8/20              int64
2/9/20              int64
2/10/20             int64
2/11/20             int64
2/12/20             int64
2/13/20             int64
2/14/20             int64
2/15/20             int64
2/16/20             int64
2/17/20             int64
2/18/20             int64
2/19/20             int64
2/20/20             int64
2/21/20             int64
2/22/20             int64
2/23/20             int64
2/24/20             int64
2/25/20     

deaths_global (254, 73)


...

Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
1/23/20             int64
1/24/20             int64
1/25/20             int64
1/26/20             int64
1/27/20             int64
1/28/20             int64
1/29/20             int64
1/30/20             int64
1/31/20             int64
2/1/20              int64
2/2/20              int64
2/3/20              int64
2/4/20              int64
2/5/20              int64
2/6/20              int64
2/7/20              int64
2/8/20              int64
2/9/20              int64
2/10/20             int64
2/11/20             int64
2/12/20             int64
2/13/20             int64
2/14/20             int64
2/15/20             int64
2/16/20             int64
2/17/20             int64
2/18/20             int64
2/19/20             int64
2/20/20             int64
2/21/20             int64
2/22/20             int64
2/23/20             int64
2/24/20             int64
2/25/20     

recovered_global (240, 73)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20
0,NaN,Afghanistan,33.000,65.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2
1,NaN,Albania,41.153,20.168,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,10,17,17,31,31,33,44
2,NaN,Algeria,28.034,1.660,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,12,12,12,12,12,32,32,32,65,65,24,65,29,29,31,31,37
3,NaN,Andorra,42.506,1.522,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,10
4,NaN,Angola,-11.203,17.874,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20
count,64,240,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000,240.000
unique,64,178,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
top,Northern Territory,China,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
freq,1,33,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
mean,NaN,NaN,20.693,29.341,0.117,0.125,0.150,0.163,0.217,0.254,0.446,0.525,0.596,0.925,1.183,1.967,2.596,3.550,4.683,6.196,8.379,10.900,13.517,16.442,19.512,21.458,26.229,33.575,39.146,45.271,52.429,59.800,67.171,75.737,78.708,95.358,97.475,105.112,116.271,126.600,138.654,152.963,165.758,177.983,190.008,200.950,213.208,224.150,232.771,243.158,252.892,260.392,268.350,279.179,284.683,292.712,302.600,316.808,325.367,336.833,347.133,354.062,364.250,382.050,407.913,409.796,450.000,474.113,508.958,545.479,580.896,621.175,685.692
std,NaN,NaN,24.180,67.952,1.807,1.811,2.007,2.075,2.720,2.928,5.195,5.714,5.876,9.204,11.015,19.235,25.195,34.117,41.586,53.740,73.288,94.760,118.086,145.955,173.226,177.616,227.505,312.023,367.054,432.756,511.882,593.516,671.825,765.721,772.778,992.577,996.044,"1,086.711","1,229.907","1,358.624","1,514.189","1,708.727","1,875.701","2,039.538","2,194.103","2,340.728","2,492.193","2,623.561","2,716.672","2,812.387","2,925.241","3,006.727","3,088.549","3,178.979","3,255.142","3,335.072","3,426.500","3,519.415","3,575.636","3,636.221","3,699.401","3,751.369","3,803.294","3,855.244","3,899.998","3,928.250","3,986.292","4,042.761","4,108.933","4,183.643","4,270.921","4,362.027","4,506.311"
min,NaN,NaN,-41.455,-106.347,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,NaN,NaN,7.405,-6.957,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0

Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
1/23/20             int64
1/24/20             int64
1/25/20             int64
1/26/20             int64
1/27/20             int64
1/28/20             int64
1/29/20             int64
1/30/20             int64
1/31/20             int64
2/1/20              int64
2/2/20              int64
2/3/20              int64
2/4/20              int64
2/5/20              int64
2/6/20              int64
2/7/20              int64
2/8/20              int64
2/9/20              int64
2/10/20             int64
2/11/20             int64
2/12/20             int64
2/13/20             int64
2/14/20             int64
2/15/20             int64
2/16/20             int64
2/17/20             int64
2/18/20             int64
2/19/20             int64
2/20/20             int64
2/21/20             int64
2/22/20             int64
2/23/20             int64
2/24/20             int64
2/25/20     

In [5]:
df_dct = dp.bulk_reduce_mem(df_dct)
# population_df = population_df.pipe(dp.reduce_mem)
gc.collect()

22

# Global measures dataframes

In [6]:
country_lvl_df_dct = dict()

for df_name in df_dct.keys():
    country_lvl_df_dct[df_name] = df_dct[df_name].drop(columns=[province_state_vname, "Lat", "Long"])
    country_lvl_df_dct[df_name] = country_lvl_df_dct[df_name].groupby(country_region_vname).sum().astype(int).reset_index(drop=False)
    country_lvl_df_dct[df_name][country_region_vname] = country_lvl_df_dct[df_name][country_region_vname].pipe(dp.normalise_str)
    exec(df_name+" = country_lvl_df_dct['"+df_name+"']")  # creates a reference for friendly_names

In [7]:
confirmed_global.head()

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20
0,AFGHANISTAN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74,84,94,110,110,120,170
1,ALBANIA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174,186,197,212,223
2,ALGERIA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,74,87,90,139,201,230,264,302,367,409,454,511,584
3,ANDORRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,39,53,75,88,113,133,164,188,224,267,308,334,370
4,ANGOLA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,4,4,5,7,7


In [8]:
population_df = population_data.copy().rename(columns={"countriesAndTerritories": country_region_vname, "popData2018": susceptible_vname})
population_df = population_df.sort_values("dateRep", ascending=False).reset_index(drop=True).drop_duplicates(country_region_vname)
population_df[country_region_vname] = population_df[country_region_vname].pipe(dp.normalise_str)
population_df.head()

,dateRep,day,month,year,cases,deaths,Country/Region,geoId,countryterritoryCode,s_0
0,2020-03-30,30,3,2020,8,1,AFGHANISTAN,AF,AFG,"37,172,386.000"
1,2020-03-30,30,3,2020,41,0,IRAQ,IQ,IRQ,"38,433,600.000"
2,2020-03-30,30,3,2020,200,10,IRELAND,IE,IRL,"4,853,506.000"
3,2020-03-30,30,3,2020,10,0,ISLE OF MAN,IM,IMN,"84,077.000"
4,2020-03-30,30,3,2020,0,0,SUDAN,SD,SDN,"41,801,533.000"


In [9]:
# Fill in missing province/state
# # Start timer
# start_time = pd.to_datetime("now")
# time_fmt = "%d/%m/%Y %H:%M:%S"
# print("INFO: start time", start_time.strftime(time_fmt))

# # Filling missing province/state where possible
# gmaps = googlemaps.Client(key=os.getenv("gmaps_api_key"))

# fillna_keys = ["Province/State", "Country/Region"]
# loc_admin_long_vname = "long_name"

# # Start with the first df with missing province/state values
# temp_df = confirmed_global[match_keys][confirmed_global[prov_state_vname].isna()].copy()

# for i in temp_df.index:
#     row = temp_df.loc[i]

#     # Look up an address with reverse geocoding
#     try:
#         target_loc = gmaps.reverse_geocode((row["Lat"], row["Long"]))[0]['address_components']
#     except IndexError:
#         continue
#     admin_ind = dp.find_admin_loc(target_loc)
#     if admin_ind is not None:
#         temp_df.loc[i, prov_state_vname] = target_loc[dp.find_admin_loc(target_loc)][loc_admin_long_vname]
    
#     # Fill missing with country/region
#     temp_df[prov_state_vname] = temp_df[prov_state_vname].fillna(temp_df[country_region_vname])
    
#     # Gmaps anti-throttling tactic
#     time.sleep(1)  # wait 1 second before the next iteration

# # Fillna missing values
# for df in df_dct.keys():
#     df_dct[df][fillna_keys] = confirmed_global[fillna_keys].fillna(temp_df[fillna_keys])

# # Finish time
# finish_time = pd.to_datetime("now")
# elapsed_time_min = round((finish_time-start_time).total_seconds()/60, 2)
# print("INFO: finished. This took", elapsed_time_min, "minutes.")

# SIR
[Source code](https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/)

In [10]:
# # Total population, N.
# N = 1000

# # Initial number of infected and recovered individuals, I0 and R0.
# I0, R0 = 1, 0

# # Everyone else, S0, is susceptible to infection initially.
# S0 = N - I0 - R0

# # Contact rate (aka beta), and mean recovery rate aka (gamma) both (in 1/days).
# beta, gamma = 0.2, 1./10

# # A grid of time points (in days)
# days = 160
# t = np.linspace(0, days, days)

# # Initial conditions vector
# y0 = S0, I0, R0
# # Integrate the SIR equations over the time grid, t.
# ret = odeint(dp.deriv, y0, t, args=(N, beta, gamma))
# S, I, R = ret.T

# # Plot the data on three separate curves for S(t), I(t) and R(t)
# fig = plt.figure(facecolor='w')
# ax = fig.add_subplot(111, axisbelow=True)
# ax.plot(t, S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
# ax.plot(t, I/1000, 'r', alpha=0.5, lw=2, label='Infected')
# ax.plot(t, R/1000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
# ax.set_facecolor('#dddddd')
# ax.set_xlabel('Time /days')
# ax.set_ylabel('Number (1000s)')
# ax.set_ylim(0,1.2)
# ax.yaxis.set_tick_params(length=0)
# ax.xaxis.set_tick_params(length=0)
# ax.grid(b=True, which='major', c='w', lw=2, ls='-')
# legend = ax.legend()
# legend.get_frame().set_alpha(0.5)
# for spine in ('top', 'right', 'bottom', 'left'):
#     ax.spines[spine].set_visible(False)
# plt.show()

## Estimating SIR beta and gamma parameters

In [11]:
countries = confirmed_global[country_region_vname].drop_duplicates()
# [confirmed_global[country_region_vname].str.contains(r"france|germany|netherlands", case=False)]

In [12]:
results_df = estimate_sir_params(countries=countries, df_dct=country_lvl_df_dct, s_0_src=population_df)

country=AFGHANISTAN, beta=0.00014568, gamma=0.00000001, r_0=14567.85509550
country=ALBANIA, beta=0.00027862, gamma=0.03472880, r_0=0.00802259
country=ALGERIA, beta=0.00022009, gamma=0.01688509, r_0=0.01303440
country=ANDORRA, beta=0.00023147, gamma=0.00019079, r_0=1.21323569
country=ANGOLA, beta=0.00018202, gamma=0.00000001, r_0=18202.21236295
country=ANTIGUA AND BARBUDA, beta=0.00012097, gamma=0.00000001, r_0=12096.83510832
country=ARGENTINA, beta=0.00032988, gamma=0.03080423, r_0=0.01070898
country=ARMENIA, beta=0.00025070, gamma=0.01074034, r_0=0.02334174
country=AUSTRALIA, beta=0.00013211, gamma=0.00802610, r_0=0.01645975
country=AUSTRIA, beta=0.00033092, gamma=0.01896900, r_0=0.01744557
country=AZERBAIJAN, beta=0.00016681, gamma=0.01297036, r_0=0.01286118
country=BAHAMAS, beta=0.00018514, gamma=0.00000001, r_0=18513.77611307
country=BAHRAIN, beta=0.00050815, gamma=0.34145026, r_0=0.00148821
country=BANGLADESH, beta=0.00015906, gamma=0.03758206, r_0=0.00423234
country=BARBADOS, bet

## Export results

In [14]:
results_df.to_csv(export_foldname+"sir_params_per_country.csv")

In [15]:
# TODO: increase estimation speed
# TODO: (future) return date or date_range with beta and gamma